# AutoML: Build Production-Ready Models Quickly!

## Classification with Porto Seguro - Part 1

https://www.linkedin.com/learning/automl-build-production-ready-models-quickly/

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
# from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [2]:
# train_data = TabularDataset('../01-machine-learning-recap/data/train_data.csv')
# train_data = TabularDataset('train_data_insure.csv')
train_data = TabularDataset("porto_train.csv")
test_data = TabularDataset("porto_test.csv")

In [3]:
train_data.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,fold
0,9,0,1,1,7,0,0,0,0,1,...,1,1,9,0,1,1,0,1,0,5
1,13,0,5,4,9,1,0,0,0,1,...,2,7,7,0,1,1,0,1,0,1
2,16,0,0,1,2,0,0,1,0,0,...,2,4,9,0,0,0,0,0,0,3
3,17,0,0,2,0,1,0,1,0,0,...,1,1,3,0,0,0,1,1,0,2
4,20,0,2,1,3,1,0,0,1,0,...,0,0,10,0,1,0,0,1,0,1


In [4]:
# train, test = train_test_split(train_data, test_size=0.3, shuffle=True)

In [5]:
train_data = train_data.replace(-1, np.nan)
test_data = test_data.replace(-1, np.nan)

In [6]:
train_data = train_data.drop(["id", "fold"], axis=1)
test_data = test_data.drop(["id"], axis=1)

In [7]:
cat_vars = [col for col in train_data.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [8]:
for col in cat_vars:
    test_data[col] = test_data[col].astype('category')
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_data[col] = train_data[col].astype('category')

## Modeling

In [9]:
# EVAL_METRIC = "f1"
EVAL_METRIC = "roc_auc"
# SAVE_PATH = "AutoGluonModels"   # Trained models will be saved here
SAVE_PATH = "AutoGluonModels_porto"   # Trained models will be saved here
# LABEL = "Claim"
LABEL = "target"

In [10]:
predictor = TabularPredictor(label=LABEL, path=SAVE_PATH, eval_metric=EVAL_METRIC)

In [12]:
# predictor.fit(train)
predictor.fit(train_data)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutoGluonModels_porto/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fit

In [13]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [14]:
predictor.leaderboard(extra_info=True)

                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order  num_features  num_models  num_models_w_ancestors  memory_size  memory_size_w_ancestors  memory_size_min  memory_size_min_w_ancestors  num_ancestors  num_descendants                  model_type             child_model_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  hyperparameters                   hyperparameters_fit  \
0   WeightedEnsemble_L2   0.611518       0.292055  

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,num_features,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,WeightedEnsemble_L2,0.611518,0.292055,330.984127,0.001172,1.381623,2,True,14,4,...,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetTorch, CatBoost, ExtraTreesEntr, XGBoost]",None,{'ensemble_size': 100},{'ensemble_size': 14},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetTorch, CatBoost, ExtraTreesEntr, XGBoost]",[]
1,CatBoost,0.602004,0.024967,54.145724,0.024967,54.145724,1,True,7,57,...,"{'iterations': 10000, 'learning_rate': 0.05, 'random_seed': 0, 'allow_writing_files': False, 'eval_metric': 'Logloss'}",{'iterations': 373},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}","[ps_ind_01, ps_ind_03, ps_ind_06_bin, ps_ind_07_bin, ps_ind_08_bin, ps_ind_09_bin, ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin, ps_ind_14, ps_ind_15, ps_ind_16_bin, ps_ind_17_bin, ps_ind_18_bin, ps_reg_01, ps_reg_02, ps_reg_03, ps_car_08_cat, ps_car_11, ps_car_12, ps_car_13, ps_car_14, ps_car_15, ps_calc_01, ps_calc_02, ps_calc_03, ps_calc_04, ps_calc_05, ps_calc_06, ps_calc_07, ps_calc_08, ps_calc_09, ps_calc_10, ps_calc_11, ps_calc_12, ps_calc_13, ps_calc_14, ps_calc_15_bin, ps_calc_16_bin, ps_calc_17_bin, ps_calc_18_bin, ps_calc_19_bin, ps_calc_20_bin, ps_ind_02_cat, ps_i...",None,None,None,None,[],[WeightedEnsemble_L2]
2,ExtraTreesEntr,0.601840,0.157045,11.755574,0.157045,11.755574,1,True,9,57,...,"{'n_estimators': 300, 'max_leaf_nodes': 15000, 'n_jobs': -1, 'random_state': 0, 'bootstrap': True, 'criterion': 'entropy'}",{'n_estimators': 300},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}","[ps_ind_01, ps_ind_03, ps_ind_06_bin, ps_ind_07_bin, ps_ind_08_bin, ps_ind_09_bin, ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin, ps_ind_14, ps_ind_15, ps_ind_16_bin, ps_ind_17_bin, ps_ind_18_bin, ps_reg_01, ps_reg_02, ps_reg_03, ps_car_08_cat, ps_car_11, ps_car_12, ps_car_13, ps_car_14, ps_car_15, ps_calc_01, ps_calc_02, ps_calc_03, ps_calc_04, ps_calc_05, ps_calc_06, ps_calc_07, ps_calc_08, ps_calc_09, ps_calc_10, ps_calc_11, ps_calc_12, ps_calc_13, ps_calc_14, ps_calc_15_bin, ps_calc_16_bin, ps_calc_17_bin, ps_calc_18_bin, ps_calc_19_bin, ps_calc_20_bin, ps_ind_02_cat, ps_i...",None,None,None,None,[],[WeightedEnsemble_L2]
3,XGBoost,0.601470,0.047181,7.777772,0.047181,7.777772,1,True,11,57,...,"{'n_estimators': 10000, 'lear

## Evaluation

In [16]:
test_leaderboard = predictor.leaderboard(
    test_data, 
    extra_metrics=["accuracy", "balanced_accuracy"], 
    silent=True, 
    extra_info=True
)

In [17]:
test_leaderboard

,model,score_test,accuracy,balanced_accuracy,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,CatBoost,0.635058,0.963609,0.500457,0.602004,0.278491,0.024967,54.145724,0.278491,0.024967,...,"{'iterations': 10000, 'learning_rate': 0.05, 'random_seed': 0, 'allow_writing_files': False, 'eval_metric': 'Logloss'}",{'iterations': 373},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}","[ps_ind_01, ps_ind_03, ps_ind_06_bin, ps_ind_07_bin, ps_ind_08_bin, ps_ind_09_bin, ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin, ps_ind_14, ps_ind_15, ps_ind_16_bin, ps_ind_17_bin, ps_ind_18_bin, ps_reg_01, ps_reg_02, ps_reg_03, ps_car_08_cat, ps_car_11, ps_car_12, ps_car_13, ps_car_14, ps_car_15, ps_calc_01, ps_calc_02, ps_calc_03, ps_calc_04, ps_calc_05, ps_calc_06, ps_calc_07, ps_calc_08, ps_calc_09, ps_calc_10, ps_calc_11, ps_calc_12, ps_calc_13, ps_calc_14, ps_calc_15_bin, ps_calc_16_bin, ps_calc_17_bin, ps_calc_18_bin, ps_calc_19_bin, ps_calc_20_bin, ps_ind_02_cat, ps_i...",None,None,None,None,[],[WeightedEnsemble_L2]
1,XGBoost,0.634824,0.963601,0.500231,0.601470,0.970946,0.047181,7.777772,0.970946,0.047181,...,"{'n_estimators': 10000, 'learning_rate': 0.1, 'n_jobs': -1, 'proc.max_category_levels': 100, 'objective': 'binary:logistic', 'booster': 'gbtree'}",{'n_estimators': 67},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}","[ps_ind_01, ps_ind_03, ps_ind_06_bin, ps_ind_07_bin, ps_ind_08_bin, ps_ind_09_bin, ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin, ps_ind_14, ps_ind_15, ps_ind_16_bin, ps_ind_17_bin, ps_ind_18_bin, ps_reg_01, ps_reg_02, ps_reg_03, ps_car_08_cat, ps_car_11, ps_car_12, ps_car_13, ps_car_14, ps_car_15, ps_calc_01, ps_calc_02, ps_calc_03, ps_calc_04, ps_calc_05, ps_calc_06, ps_calc_07, ps_calc_08, ps_calc_09, ps_calc_10, ps_calc_11, ps_calc_12, ps_calc_13, ps_calc_14, ps_calc_15_bin, ps_calc_16_bin, ps_calc_17_bin, ps_calc_18_bin, ps_calc_19_bin, ps_calc_20_bin, ps_ind_02_cat, ps_i...",None,None,None,None,[],[WeightedEnsemble_L2]
2,WeightedEnsemble_L2,0.631491,0.963584,0.500000,0.611518,3.419083,0.292055,330.984127,0.008947,0.001172,...,"{'use_orig_features': False, 'max_base_models': 25, 'max_base_models_per_type': 5, 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetTorch, CatBoost, ExtraTreesEntr, XGBoost]",None,{'ensemble_size': 100},{'ensemble_size': 14},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[NeuralNetTorch, CatBoost, ExtraTreesEntr, XGBoost]",[]
3,Ne

In [20]:
y_pred_proba = predictor.predict_proba(test_data.drop(LABEL, axis=1))
perf = predictor.evaluate_predictions(y_true=test_data[LABEL], y_pred=y_pred_proba)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6314907429408757
Evaluations on test data:
{
    "roc_auc": 0.6314907429408757,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}
